In [ ]:
import re
import pandas as pd
import csv
import numpy as np
from collections import Counter
import nltk
import random
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import IsolationForest
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings

warnings.filterwarnings(action = 'ignore')
import gensim
from gensim.models import Word2Vec

In [ ]:
import tensorflow as tf
import math
import os
import time
import numpy as np
import pandas as pd
import random
import statistics
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential, regularizers
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Input, LSTM, Dropout, TimeDistributed, RepeatVector, Bidirectional
from tensorflow.keras.losses import CategoricalCrossentropy, MeanSquaredLogarithmicError
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Accuracy, Precision, AUC, Recall
from sklearn.metrics import silhouette_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_outlier = pd.read_csv('/content/drive/MyDrive/thesis/val/raw_val_all.csv', sep=',')

In [ ]:
df_validate = pd.read_csv('/content/drive/MyDrive/thesis/val/raw_val.csv', sep=',')

In [ ]:
df_oh = pd.read_csv('/content/drive/MyDrive/thesis/train/w2vec.csv', sep=',')
x_out_oh = pd.read_csv('/content/drive/MyDrive/thesis/val/val-w2vec.csv', sep=',')

In [ ]:
df_oh.head(5)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.101784,0.849301,1.000000,0.000000,1.00000,0.000000,0.936119,0.715389,0.808444,0.000000,...,0.173194,0.572735,0.562702,0.897191,0.335552,0.795203,0.547535,0.732336,1.000000,0.241742
1,0.101784,0.849301,1.000000,0.000000,1.00000,0.000000,0.936119,0.715389,0.808444,0.000000,...,0.173194,0.572735,0.562702,0.897191,0.335552,0.795203,0.547535,0.732336,1.000000,0.241742
2,0.101784,0.849301,1.000000,0.000000,1.00000,0.000000,0.936119,0.715389,0.808444,0.000000,...,0.173194,0.572735,0.562702,0.897191,0.335552,0.795203,0.547535,0.732336,1.000000,0.241742
3,0.101784,0.849301,1.000000,0.000000,1.00000,0.000000,0.936119,0.715389,0.808444,0.000000,...,0.173194,0.572735,0.562702,0.897191,0.335552,0.795203,0.547535,0.732336,1.000000,0.241742
4,0.222365,0.620606,0.935762,0.172489,0.63881,0.025653,0.991941,0.708340,0.700041,0.277449,...,0.665805,0.522102,0.399594,0.653415,0.159163,1.000000,0.000000,0.949909,0.892642,0.000000


In [ ]:
test = df_oh[:int(split_len)]
train = df_oh[int(split_len):]

In [ ]:
true_labels = df_outlier.label

In [ ]:
x_train = train.values.reshape(train.shape[0], 1, train.shape[1])
x_test = test.values.reshape(test.shape[0], 1,train.shape[1])

x_out = x_out_oh.values.reshape(x_out_oh.shape[0], 1,x_out_oh.shape[1])

#define the model

def autoencoder_model(X):

  inputs = Input(shape=(X.shape[1], X.shape[2]))
  L1 = LSTM(128, activation='relu', return_sequences=True,
              kernel_regularizer=regularizers.l2(0.00))(inputs)
  L2 = LSTM(64, activation='relu', return_sequences=False)(L1)
  L3 = RepeatVector(X.shape[1])(L2)
  L4 = LSTM(64, activation='relu', return_sequences=True)(L3)
  L5 = LSTM(128, activation='relu', return_sequences=True)(L4)
  output = TimeDistributed(Dense(X.shape[2]))(L5)
  model = Model(inputs=inputs, outputs=output)
  return model

# create the autoencoder model
model = autoencoder_model(x_train)
model.compile(optimizer='adam', loss=['mse'],
              metrics=['mae', 'acc', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
model.summary()

nb_epochs = 10
batch_size = 1000
history = model.fit(x_train, x_train, epochs=nb_epochs, batch_size=batch_size,
                    validation_split=0.15).history





Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 300)]          0         
                                                                 
 lstm (LSTM)                 (None, 1, 128)            219648    
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 repeat_vector (RepeatVector  (None, 1, 64)            0         
 )                                                               
                                                                 
 lstm_2 (LSTM)               (None, 1, 64)             33024     
                                                                 
 lstm_3 (LSTM)               (None, 1, 128)            98816     
                                                             

In [ ]:
# calculate the loss on the test set
X_pred = model.predict(x_out)
X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[2])
X_pred = pd.DataFrame(X_pred, columns=x_out_oh.columns)
X_pred.index = x_out_oh.index

scored = pd.DataFrame(index=x_out_oh.index)
Xtest = x_out_oh.values.reshape(x_out_oh.shape[0], x_out_oh.shape[1])
scored['Loss_mae'] = np.mean(np.abs(X_pred-Xtest), axis = 1)
scored['Threshold'] = 0.1
scored['Anomaly'] = np.where(scored['Loss_mae'] > scored['Threshold'], -1, 1)
scored.head()
test_pred = scored.Anomaly

from sklearn.metrics import classification_report
print(classification_report(true_labels, test_pred))

3188/3188 [==============================] - 8s 2ms/step
              precision    recall  f1-score   support

          -1       0.99      0.44      0.61      3737
           1       0.98      1.00      0.99     98263

    accuracy                           0.98    102000
   macro avg       0.98      0.72      0.80    102000
weighted avg       0.98      0.98      0.98    102000



In [ ]:
pip install pyod

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-1.0.9-py3-none-any.whl size=184097 sha256=97bc3e078d40f1e1f3f8ee250a275cc9fb9ee867a47cbb6de082c71e3d35dbd2
  Stored in directory: /root/.cache/pip/wheels/83/55/6b/552e083cf5509c0afe808b76cf434f1be284d01a112623bd37
Successfully built pyod


In [ ]:
true_labels = df_outlier.label

In [ ]:
from pyod.models.copod import COPOD
if __name__ == "__main__":

    # train COPOD detector
    clf_name = 'COPOD'
    clf = COPOD(contamination=0.05)

    # you could try parallel version as well.
    # clf = COPOD(n_jobs=2)
    clf.fit(train)
    pred_test = clf.predict(x_out_oh)

if (pred_test == 1).any():
    pred_test[pred_test == 1] = -1

if (pred_test == 0).any():
    pred_test[pred_test == 0] = 1

Counter(pred_test)

from sklearn.metrics import classification_report
print(classification_report(true_labels, pred_test))

              precision    recall  f1-score   support

          -1       0.30      0.62      0.40      3737
           1       0.98      0.95      0.96     98263

    accuracy                           0.93    102000
   macro avg       0.64      0.78      0.68    102000
weighted avg       0.96      0.93      0.94    102000



In [ ]:
from pyod.models.ecod import ECOD
if __name__ == "__main__":

    # train COPOD detector
    clf_name = 'ECOD'
    clf = ECOD(contamination=0.035)

    # you could try parallel version as well.
    # clf = COPOD(n_jobs=2)
    clf.fit(train)
    pred_test = clf.predict(x_out_oh)

if (pred_test == 1).any():
    pred_test[pred_test == 1] = -1

if (pred_test == 0).any():
    pred_test[pred_test == 0] = 1

Counter(pred_test)

from sklearn.metrics import classification_report
print(classification_report(true_labels, pred_test))

              precision    recall  f1-score   support

          -1       0.47      0.62      0.53      3737
           1       0.99      0.97      0.98     98263

    accuracy                           0.96    102000
   macro avg       0.73      0.79      0.76    102000
weighted avg       0.97      0.96      0.96    102000



In [ ]:
clf = IsolationForest(n_estimators=200, contamination = 0.025)

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
clf_lof = LocalOutlierFactor(novelty=True, n_neighbors=15, contamination=0.065)

In [ ]:
clf_if_tf = clf_lof.fit(train)

In [ ]:
from joblib import dump, load

# save the model
dump(clf_if_tf,'/content/drive/MyDrive/thesis/models/lof/clf_lof_w2v.joblib')


['/content/drive/MyDrive/thesis/models/lof/clf_lof_w2v.joblib']

In [ ]:
clf_if_tf = load('/content/drive/MyDrive/thesis/models/lof/clf_lof_w2v.joblib')

In [ ]:
pred_test = clf_if_tf.predict(x_out_oh)

In [ ]:
Counter(pred_test)

Counter({1: 101854, -1: 146})

In [ ]:
true_labels = df_outlier.label

In [ ]:
df_validate['label'] = pred_test

In [ ]:
df_validate[df_validate['label'] == -1].to_csv('/content/drive/MyDrive/thesis/models/iforest/w2v_results_val.csv', index=False)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(true_labels, pred_test))

              precision    recall  f1-score   support

          -1       0.98      0.04      0.07      3737
           1       0.96      1.00      0.98     98263

    accuracy                           0.96    102000
   macro avg       0.97      0.52      0.53    102000
weighted avg       0.97      0.96      0.95    102000



In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming you have the true labels for the dataset in 'true_labels' and the LOF predictions in 'lof_predictions'

# Convert LOF scores to binary labels based on a threshold (e.g., outliers with scores above the threshold are considered anomalies)

# Calculate accuracy
accuracy = accuracy_score(true_labels, pred_test)
print("Accuracy:", accuracy)

# Calculate precision
precision = precision_score(true_labels, pred_test)
print("Precision:", precision)

# Calculate recall
recall = recall_score(true_labels, pred_test)
print("Recall:", recall)

# Calculate F1 score
f1 = f1_score(true_labels, pred_test)
print("F1 Score:", f1)

Accuracy: 0.9690196078431372
Precision: 0.9774199054226363
Recall: 0.9907289620711763
F1 Score: 0.9840294343589537


W2V+PCA

In [ ]:
df_oh = pd.read_csv('/content/drive/MyDrive/thesis/train/w2vec.csv', sep=',')
x_out_oh = pd.read_csv('/content/drive/MyDrive/thesis/val/val-w2vec.csv', sep=',')

In [ ]:
split_len = 0.3 * len(df_oh)
split_len

306302.7

In [ ]:
test = df_oh[:int(split_len)]
train = df_oh[int(split_len):]

In [ ]:
x_train = train.values.reshape(train.shape[0], 1, train.shape[1])
x_test = test.values.reshape(test.shape[0], 1,train.shape[1])

x_out = x_out_oh.values.reshape(x_out_oh.shape[0], 1,x_out_oh.shape[1])

#define the model

def autoencoder_model(X):

  inputs = Input(shape=(X.shape[1], X.shape[2]))
  L1 = Bidirectional(LSTM(128, activation='relu', return_sequences=True,
              kernel_regularizer=regularizers.l2(0.00)))(inputs)
  L2 = Bidirectional(LSTM(64, activation='relu', return_sequences=False))(L1)
  L3 = RepeatVector(X.shape[1])(L2)
  L4 = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(L3)
  L5 = Bidirectional(LSTM(128, activation='relu', return_sequences=True))(L4)
  output = TimeDistributed(Dense(X.shape[2]))(L5)
  model = Model(inputs=inputs, outputs=output)
  return model

# create the autoencoder model
model = autoencoder_model(x_train)
model.compile(optimizer='adam', loss=['mse'],
              metrics=['mae', 'acc', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
model.summary()

nb_epochs = 10
batch_size = 1000
history = model.fit(x_train, x_train, epochs=nb_epochs, batch_size=batch_size,
                    validation_split=0.15).history





Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1, 300)]          0         
                                                                 
 bidirectional (Bidirectiona  (None, 1, 256)           439296    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 repeat_vector_2 (RepeatVect  (None, 1, 128)           0         
 or)                                                             
                                                                 
 bidirectional_2 (Bidirectio  (None, 1, 128)           98816     
 nal)                                                      

In [ ]:
# calculate the loss on the test set
X_pred = model.predict(x_out)
X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[2])
X_pred = pd.DataFrame(X_pred, columns=x_out_oh.columns)
X_pred.index = x_out_oh.index

scored = pd.DataFrame(index=x_out_oh.index)
Xtest = x_out_oh.values.reshape(x_out_oh.shape[0], x_out_oh.shape[1])
scored['Loss_mae'] = np.mean(np.abs(X_pred-Xtest), axis = 1)
scored['Threshold'] = 0.004
scored['Anomaly'] = np.where(scored['Loss_mae'] > scored['Threshold'], -1, 1)
scored.head()
test_pred = scored.Anomaly

from sklearn.metrics import classification_report
print(classification_report(true_labels, test_pred))

3188/3188 [==============================] - 11s 3ms/step
              precision    recall  f1-score   support

          -1       0.74      0.62      0.67      3737
           1       0.99      0.99      0.99     98263

    accuracy                           0.98    102000
   macro avg       0.86      0.80      0.83    102000
weighted avg       0.98      0.98      0.98    102000



In [ ]:
from pyod.models.copod import COPOD
if __name__ == "__main__":

    # train COPOD detector
    clf_name = 'COPOD'
    clf = COPOD(contamination=0.04)

    # you could try parallel version as well.
    # clf = COPOD(n_jobs=2)
    clf.fit(train)
    pred_test = clf.predict(x_out_oh)

if (pred_test == 1).any():
    pred_test[pred_test == 1] = -1

if (pred_test == 0).any():
    pred_test[pred_test == 0] = 1

Counter(pred_test)

from sklearn.metrics import classification_report
print(classification_report(true_labels, pred_test))

              precision    recall  f1-score   support

          -1       0.47      0.62      0.53      3737
           1       0.99      0.97      0.98     98263

    accuracy                           0.96    102000
   macro avg       0.73      0.79      0.76    102000
weighted avg       0.97      0.96      0.96    102000



In [ ]:
from pyod.models.ecod import ECOD
if __name__ == "__main__":

    # train COPOD detector
    clf_name = 'ECOD'
    clf = ECOD(contamination=0.020)

    # you could try parallel version as well.
    # clf = COPOD(n_jobs=2)
    clf.fit(train)
    pred_test = clf.predict(x_out_oh)

if (pred_test == 1).any():
    pred_test[pred_test == 1] = -1

if (pred_test == 0).any():
    pred_test[pred_test == 0] = 1

Counter(pred_test)

from sklearn.metrics import classification_report
print(classification_report(true_labels, pred_test))

              precision    recall  f1-score   support

          -1       0.85      0.62      0.71      3737
           1       0.99      1.00      0.99     98263

    accuracy                           0.98    102000
   macro avg       0.92      0.81      0.85    102000
weighted avg       0.98      0.98      0.98    102000



In [ ]:
clf = IsolationForest(n_estimators=200, contamination = 0.025)

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
clf_lof = LocalOutlierFactor(novelty=True, n_neighbors=15, contamination=0.065)

In [ ]:
clf_if_tf = clf_lof.fit(train)

In [ ]:
from joblib import dump, load

# save the model
dump(clf_if_tf,'/content/drive/MyDrive/thesis/models/lof/clf_lof_w2v_pca.joblib')

['/content/drive/MyDrive/thesis/models/lof/clf_lof_w2v_pca.joblib']

In [ ]:
clf_if_tf = load('/content/drive/MyDrive/thesis/models/lof/clf_lof_w2v_pca.joblib')

In [ ]:
pred_test = clf_if_tf.predict(x_out_oh)

In [ ]:
Counter(pred_test)

Counter({1: 101854, -1: 146})

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(true_labels, pred_test))

              precision    recall  f1-score   support

          -1       0.98      0.04      0.07      3737
           1       0.96      1.00      0.98     98263

    accuracy                           0.96    102000
   macro avg       0.97      0.52      0.53    102000
weighted avg       0.97      0.96      0.95    102000



In [ ]:
true_labels = df_outlier.label

In [ ]:
df_validate['label'] = pred_test

In [ ]:
df_validate[df_validate['label'] == -1].to_csv('/content/drive/MyDrive/thesis/models/iforest/w2v_results_val_pca.csv', index=False)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(true_labels, pred_test))

              precision    recall  f1-score   support

          -1       0.81      0.80      0.80      3737
           1       0.99      0.99      0.99     98263

    accuracy                           0.99    102000
   macro avg       0.90      0.89      0.90    102000
weighted avg       0.99      0.99      0.99    102000



In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming you have the true labels for the dataset in 'true_labels' and the LOF predictions in 'lof_predictions'

# Convert LOF scores to binary labels based on a threshold (e.g., outliers with scores above the threshold are considered anomalies)

# Calculate accuracy
accuracy = accuracy_score(true_labels, pred_test)
print("Accuracy:", accuracy)

# Calculate precision
precision = precision_score(true_labels, pred_test)
print("Precision:", precision)

# Calculate recall
recall = recall_score(true_labels, pred_test)
print("Recall:", recall)

# Calculate F1 score
f1 = f1_score(true_labels, pred_test)
print("F1 Score:", f1)

Accuracy: 0.9854607843137255
Precision: 0.9922385205379123
Recall: 0.9926727252373732
F1 Score: 0.992455575395917


W2v+AE

In [ ]:
df_oh = pd.read_csv('/content/drive/MyDrive/thesis/train/w2vec-reduce.csv', sep=',')
x_out_oh = pd.read_csv('/content/drive/MyDrive/thesis/val/val-w2vec-reduce.csv', sep=',')

In [ ]:
df_oh.head(5)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.141374,0.515627,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.141374,0.515627,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.141374,0.515627,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.141374,0.515627,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.067468,0.605815,0.0,0.0,0.0,0.741926,0.0,0.0,1.610108


In [ ]:
from sklearn.preprocessing import MinMaxScaler

mm_scaler = MinMaxScaler()
x_scaled = mm_scaler.fit_transform(df_oh)

In [ ]:
x = mm_scaler.transform(x_out_oh)

In [ ]:
df_oh = pd.DataFrame(x_scaled)
x_out_oh =  pd.DataFrame(x)

In [ ]:
test = df_oh[:int(split_len)]
train = df_oh[int(split_len):]

In [ ]:
x_train = train.values.reshape(train.shape[0], 1, train.shape[1])
x_test = test.values.reshape(test.shape[0], 1,train.shape[1])

x_out = x_out_oh.values.reshape(x_out_oh.shape[0], 1,x_out_oh.shape[1])

#define the model

def autoencoder_model(X):

  inputs = Input(shape=(X.shape[1], X.shape[2]))
  L1 = Bidirectional(LSTM(128, activation='relu', return_sequences=True,
              kernel_regularizer=regularizers.l2(0.00)))(inputs)
  L2 = Bidirectional(LSTM(64, activation='relu', return_sequences=False))(L1)
  L3 = RepeatVector(X.shape[1])(L2)
  L4 = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(L3)
  L5 = Bidirectional(LSTM(128, activation='relu', return_sequences=True))(L4)
  output = TimeDistributed(Dense(X.shape[2]))(L5)
  model = Model(inputs=inputs, outputs=output)
  return model


# create the autoencoder model
model = autoencoder_model(x_train)
model.compile(optimizer='adam', loss=['mse'],
              metrics=['mae', 'acc', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
model.summary()

nb_epochs = 10
batch_size = 1000
history = model.fit(x_train, x_train, epochs=nb_epochs, batch_size=batch_size,
                    validation_split=0.15).history





Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1, 32)]           0         
                                                                 
 bidirectional_4 (Bidirectio  (None, 1, 256)           164864    
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 repeat_vector_3 (RepeatVect  (None, 1, 128)           0         
 or)                                                             
                                                                 
 bidirectional_6 (Bidirectio  (None, 1, 128)           98816     
 nal)                                                      

In [ ]:
# calculate the loss on the test set
X_pred = model.predict(x_out)
X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[2])
X_pred = pd.DataFrame(X_pred, columns=x_out_oh.columns)
X_pred.index = x_out_oh.index

scored = pd.DataFrame(index=x_out_oh.index)
Xtest = x_out_oh.values.reshape(x_out_oh.shape[0], x_out_oh.shape[1])
scored['Loss_mae'] = np.mean(np.abs(X_pred-Xtest), axis = 1)
scored['Threshold'] = 0.008
scored['Anomaly'] = np.where(scored['Loss_mae'] > scored['Threshold'], -1, 1)
scored.head()
test_pred = scored.Anomaly

from sklearn.metrics import classification_report
print(classification_report(true_labels, test_pred))

3188/3188 [==============================] - 10s 3ms/step
              precision    recall  f1-score   support

          -1       0.93      0.54      0.69      3737
           1       0.98      1.00      0.99     98263

    accuracy                           0.98    102000
   macro avg       0.95      0.77      0.84    102000
weighted avg       0.98      0.98      0.98    102000



In [ ]:
from pyod.models.copod import COPOD
if __name__ == "__main__":

    # train COPOD detector
    clf_name = 'COPOD'
    clf = COPOD(contamination=0.04)

    # you could try parallel version as well.
    # clf = COPOD(n_jobs=2)
    clf.fit(train)
    pred_test = clf.predict(x_out_oh)

if (pred_test == 1).any():
    pred_test[pred_test == 1] = -1

if (pred_test == 0).any():
    pred_test[pred_test == 0] = 1

Counter(pred_test)

from sklearn.metrics import classification_report
print(classification_report(true_labels, pred_test))

              precision    recall  f1-score   support

          -1       0.48      0.62      0.54      3737
           1       0.99      0.97      0.98     98263

    accuracy                           0.96    102000
   macro avg       0.73      0.80      0.76    102000
weighted avg       0.97      0.96      0.96    102000



In [ ]:
from pyod.models.ecod import ECOD
if __name__ == "__main__":

    # train COPOD detector
    clf_name = 'ECOD'
    clf = ECOD(contamination=0.020)

    # you could try parallel version as well.
    # clf = COPOD(n_jobs=2)
    clf.fit(train)
    pred_test = clf.predict(x_out_oh)

if (pred_test == 1).any():
    pred_test[pred_test == 1] = -1

if (pred_test == 0).any():
    pred_test[pred_test == 0] = 1

Counter(pred_test)

from sklearn.metrics import classification_report
print(classification_report(true_labels, pred_test))

              precision    recall  f1-score   support

          -1       0.58      0.54      0.56      3737
           1       0.98      0.99      0.98     98263

    accuracy                           0.97    102000
   macro avg       0.78      0.76      0.77    102000
weighted avg       0.97      0.97      0.97    102000



In [ ]:
from sklearn.neighbors import LocalOutlierFactor
clf_lof = LocalOutlierFactor(novelty=True, n_neighbors=15, contamination=0.045)

In [ ]:
clf = IsolationForest(n_estimators=200, contamination = 0.045)

In [ ]:
clf_if_tf = clf_lof.fit(train)

In [ ]:
from joblib import dump, load

# save the model
dump(clf_if_tf,'/content/drive/MyDrive/thesis/models/lof/clf_lof_w2v_ae.joblib')

['/content/drive/MyDrive/thesis/models/lof/clf_lof_w2v_ae.joblib']

In [ ]:
clf_if_tf = load('/content/drive/MyDrive/thesis/models/lof/clf_lof_w2v_ae.joblib')

In [ ]:
pred_test = clf_if_tf.predict(x_out_oh)

In [ ]:
Counter(pred_test)

Counter({1: 101870, -1: 130})

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(true_labels, pred_test))

              precision    recall  f1-score   support

          -1       0.98      0.03      0.07      3737
           1       0.96      1.00      0.98     98263

    accuracy                           0.96    102000
   macro avg       0.97      0.52      0.52    102000
weighted avg       0.97      0.96      0.95    102000



In [ ]:
true_labels = df_outlier.label

In [ ]:
df_validate['label'] = pred_test

In [ ]:
df_validate[df_validate['label'] == -1].to_csv('/content/drive/MyDrive/thesis/models/iforest/w2v_results_val_ae.csv', index=False)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(true_labels, pred_test))

              precision    recall  f1-score   support

          -1       0.59      0.91      0.71      3737
           1       1.00      0.98      0.99     98263

    accuracy                           0.97    102000
   macro avg       0.79      0.94      0.85    102000
weighted avg       0.98      0.97      0.98    102000



In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming you have the true labels for the dataset in 'true_labels' and the LOF predictions in 'lof_predictions'

# Convert LOF scores to binary labels based on a threshold (e.g., outliers with scores above the threshold are considered anomalies)

# Calculate accuracy
accuracy = accuracy_score(true_labels, pred_test)
print("Accuracy:", accuracy)

# Calculate precision
precision = precision_score(true_labels, pred_test)
print("Precision:", precision)

# Calculate recall
recall = recall_score(true_labels, pred_test)
print("Recall:", recall)

# Calculate F1 score
f1 = f1_score(true_labels, pred_test)
print("F1 Score:", f1)

Accuracy: 0.9734607843137255
Precision: 0.9963329247262679
Recall: 0.9760438822344117
F1 Score: 0.9860840508932013
